# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking your current working directory
print(os.getcwd())
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list read csv file
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
         # Extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# Print total number of rows 
print(len(full_data_rows_list))
# Print first 2 rows of list of event data
print(full_data_rows_list[:2])

8056
[['Stephen Lynch', 'Logged In', 'Jayden', 'M', '0', 'Bell', '182.85669', 'free', 'Dallas-Fort Worth-Arlington, TX', 'PUT', 'NextSong', '1.54099E+12', '829', "Jim Henson's Dead", '200', '1.54354E+12', '91'], ['Manowar', 'Logged In', 'Jacob', 'M', '0', 'Klein', '247.562', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '1049', 'Shell Shock', '200', '1.54354E+12', '73']]


In [4]:
# Creating a smaller event data csv file that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0],
                         row[2],
                         row[3],
                         row[4],
                         row[5],
                         row[6],
                         row[7],
                         row[8],
                         row[12],
                         row[13],
                         row[16],
                         ))

In [5]:
# Check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


**The newly created event_datafile_new.csv contains the following columns:** 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data appears like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="pics/image_event_datafile_new.jpg">

# Part II. Creating Apache Cassandra Keyspace for Analytics 

#### Creating a Cluster

In [6]:
# This will make a connection to a Cassandra instance on a local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create & set Keyspace

In [7]:
# Create a Keyspace
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS analytics
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """) 

# Set KEYSPACE to the session specified above
session.set_keyspace('analytics')

## Create tables and queries to ask three questions of the data  

### Table / Query 1

**Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4**

In [8]:
query = "DROP TABLE IF EXISTS session_analytics "
session.execute(query)

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # Assign the INSERT statements into the `query` variable
        query = "CREATE TABLE IF NOT EXISTS session_analytics "
        query = query + """(session_id int, item_in_session int, artist varchar, songtitle varchar, length float, 
                           PRIMARY KEY (session_id, item_in_session))"""
        session.execute(query)
        
        # Assign which column element should be assigned for each column in the INSERT statement.
        query = "INSERT INTO session_analytics (session_id, item_in_session, artist, songtitle, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Validate

In [15]:
# Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, songtitle, length FROM session_analytics WHERE session_id = 338 AND item_in_session = 4"
result = session.execute(query)
for row in result:
    print(*row)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query / Table 2

**Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182**

**Note:** This works when we set `PRIMARY KEY (user_id, session_id, item_in_session)` but since we are retrieving data by userid and sessionid, it is better to use both of them as the partition key. Using more than one column for the partition key (such as (userId, sessionId)) breaks the data into chunks, or buckets. This will make the identification process more direct, i.e. more efficient. Here is a good [stackoverflow](https://stackoverflow.com/questions/24949676/difference-between-partition-key-composite-key-and-clustering-key-in-cassandra) discussion on these concepts. 

In [10]:
query = "DROP TABLE IF EXISTS user_analytics "
session.execute(query)

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # Assign the INSERT statements into the `query` variable
        query = "CREATE TABLE IF NOT EXISTS user_analytics "
        query = query + """(user_id int, session_id int, item_in_session int, artist varchar, songtitle varchar, user_fn varchar, user_ln varchar, 
                           PRIMARY KEY ((user_id, session_id), item_in_session))"""
        session.execute(query)
        
        # Assign which column element should be assigned for each column in the INSERT statement.
        query = "INSERT INTO user_analytics (user_id, session_id, item_in_session, artist, songtitle, user_fn, user_ln)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Validate

In [14]:
# Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, songtitle, user_fn, user_ln FROM user_analytics WHERE user_id = 10 AND session_id = 182"
result = session.execute(query)
for row in result:
    print(*row)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query / Table 3

**Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'**

**Note:** This is a tricky one - if you'd only use `songtitle` as PRIMARY KEY, you'd get a result, but only one row - because Cassandra does not allow duplicate keys. To get more than one result you have to make sure there is no duplicated PRIMARY KEY values, even if more than one person has listended to the song. Hence you have to add `user_id`.

In [12]:
query = "DROP TABLE IF EXISTS song_analytics "
session.execute(query)

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # Assign the INSERT statements into the `query` variable
        query = "CREATE TABLE IF NOT EXISTS song_analytics "
        query = query + """(songtitle varchar, user_id int, user_fn varchar, user_ln varchar, 
                           PRIMARY KEY (songtitle, user_id))"""
        session.execute(query)
        
        # Assign which column element should be assigned for each, column in the INSERT statement.
        query = "INSERT INTO song_analytics (songtitle, user_id, user_fn, user_ln)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Validate

In [16]:
# Add in the SELECT statement to verify the data was entered into the table
query = "SELECT user_fn, user_ln FROM song_analytics WHERE songtitle = 'All Hands Against His Own'"
result = session.execute(query)
for row in result:
    print(*row)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [19]:
for table in ["session_analytics", "user_analytics", "song_analytics"]:
    query = f"DROP TABLE IF EXISTS {table}"
    session.execute(query)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()

---